## OLP_LLM: Pipeline for generating FOON graphs via LLM Prompting

## Step 1 :- Generate a FOON via LLM Prompting

1. Initialize libraries needed for FOON (``FOON_graph_analyser.py``) as well as OpenAI api.

2. Perform 2-stage prompting for recipe prototype.
    - In the first stage, we ask the LLM for a *high-level recipe* (list of instructions) and a *list of objects* needed for completing the recipe.
    - In the second stage, we ask the LLM for a breakdown of *state changes* that happen for each step of the recipe; specifically, we ask for the *preconditions* and *effects* of each action, which is similar to how a functional unit in FOON has *input* and *output object nodes*.

### Initialize library

In [1]:
from olp_library import *

### Language to  OLP

In [2]:
openai_models = ["gpt-3.5-turbo", "gpt-4-1106-preview", "gpt-4",]

# NOTE: all incontext examples will be stored within a JSON file:
# -- Q: can we randomly sample from the set of incontext examples?
# -- Q: should we also select an example "closest" to the provided task?
# incontext_file = "incontext_examples.txt"
incontext_file = "incontext_examples.json"

LLM_to_PDDL = True

user_tasks = [
    "I am thirsty and in need of energy: how do I make coffee?",
    "I am thirsty: how do I make a lemonade?",
    "I am thirsty: how do I make a Bloody Mary cocktail?",
    # "I have a red block, a green block, and a blue block on a table: how do I stack them such that the colours are in alphabetical order?",
    # "I have a red block, a green block, and a blue block. and a box on a table: how do I pack the blocks into the box such that the colours are in alphabetical order?",
	"I am packing Christmas gifts. I have 2 white toys, 1 red toy, and 3 green toys. I also have a white box and a green box."\
        " Which toys can I pack such that each box contains toys of the same colour?"
]

user_task = user_tasks[-1]

llm_output = generate_OLP(user_task, incontext_file, openai_models[-1], verbose=False)

print('Objects used in plan:', llm_output['RelevantObjects'])

json.dump(llm_output['OLP'], open('raw_OLP.json', 'w'), indent=4)

[2, 5] 

Objects used in plan: ['first white toy', 'second white toy', 'white box', 'first green toy', 'second green toy', 'third green toy', 'green box']


In [3]:
## Visualize Outputs
print(f"***********************\n    High level plan\n***********************\n{llm_output['PlanSketch']}\n")
print("***********************\n   Object level plan\n***********************\n")
print(llm_output['OLP'], sep="\n")
print(f"\n***********************\n   Plan objects\n***********************\n{llm_output['RelevantObjects']}\n")

***********************
    High level plan
***********************
High-Level Plan:
1. Pick and place the first white toy in the white box.
2. Pick and place the second white toy in the white box.
3. Pick and place the first green toy in the green box.
4. Pick and place the second green toy in the green box.
5. Pick and place the third green toy in the green box.

unique_objects: ["first white toy", "second white toy", "white box", "first green toy", "second green toy", "third green toy", "green box"]

***********************
   Object level plan
***********************

{'Query': 'Packing Christmas gifts with same color toys in same color boxes', 'CompleteObjectSet': ['first white toy', 'second white toy', 'white box', 'first green toy', 'second green toy', 'third green toy', 'green box'], 'Instructions': [{'Step': 1, 'Instruction': 'Pick and place the first white toy in the white box.', 'RelatedObjects': ['first white toy', 'white box'], 'Action': 'Pick and Place', 'State': {'first 

#### Sample FOON creation


In [4]:
# plan_step, plan_objects = llm_output['OLP'],

if '.json' in str(incontext_file).lower():
    sample_unit = create_functionalUnit_ver2(llm_output, index=0)
else:
    sample_unit = create_functionalUnit_ver1(llm_output['OLP'][0]['Instructions'],llm_output['RelevantObjects'])

sample_unit.print_functions[2](version=1)

Creating functional unit for Step 1: Pick and place the first white toy in the white box.
-> related objects: ['first white toy', 'white box']
Current object is : first white toy
	 first white toy state changes: {'Precondition': ['on table'], 'Effect': ['in white box']}
		 Precondition : on table || related objects: table
		 Effect : in white box || related objects: white box
Current object is : white box
	 white box state changes: {'Precondition': ['empty'], 'Effect': ['contains first white toy']}
		 Precondition : empty || related objects: None
		 Effect : contains first white toy || related objects: None
O	first white toy
S	on	 [table]
O	white box
S	empty
M	Pick and Place	<Assumed>
O	first white toy
S	in	 [white box]
O	white box
S	contains	{first white toy}


#### Creating FOON units

In [5]:
FOON_prototype = []

for x in range(len(llm_output['OLP']['Instructions'])):

    new_unit = None

    if '.json' in str(incontext_file).lower():
        new_unit = create_functionalUnit_ver2(llm_output, index=x)

    else:
        print(step['Step'])
        # -- now we will create functional units that follow the FOON format:
        new_unit = create_functionalUnit_ver1(step, plan_objects)

    # NOTE: in order to define a macro-problem, we need to properly identify all goal nodes;
    #       we will do this with the help of the LLM:
    if 'TerminalSteps' in llm_output and x in llm_output['TerminalSteps']:
        # -- set output objects as goal nodes for the functional units deemed as terminal steps:
        for N in range(new_unit.getNumberOfOutputs()):
            new_unit.getOutputNodes()[N].setAsGoal()

    elif x == (len(llm_output['OLP']['Instructions']) - 1):
        # -- by default, treat the last functional unit as the terminal unit:
        for N in range(new_unit.getNumberOfOutputs()):
            new_unit.getOutputNodes()[N].setAsGoal()

    # -- add the functional unit to the FOON prototype:

    if not new_unit.isEmpty():
        # -- we should only add a new functional unit if it is not empty, meaning it must have the following:
        #    1. >=1 input node and >= 1 output node
        #    2. a valid motion node
        FOON_prototype.append(new_unit)
    else:
        print('NOTE: the following functional unit has an error, so skipping it:')
        new_unit.print_functions[-1]()

    print()


Creating functional unit for Step 1: Pick and place the first white toy in the white box.
-> related objects: ['first white toy', 'white box']
Current object is : first white toy
	 first white toy state changes: {'Precondition': ['on table'], 'Effect': ['in white box']}
		 Precondition : on table || related objects: table
		 Effect : in white box || related objects: white box
Current object is : white box
	 white box state changes: {'Precondition': ['empty'], 'Effect': ['contains first white toy']}
		 Precondition : empty || related objects: None
		 Effect : contains first white toy || related objects: None

Creating functional unit for Step 2: Pick and place the second white toy in the white box.
-> related objects: ['second white toy', 'white box']
Current object is : second white toy
	 second white toy state changes: {'Precondition': ['on table'], 'Effect': ['in white box']}
		 Precondition : on table || related objects: table
		 Effect : in white box || related objects: white box
C

#### Testing PDDL Operator Generation via LLM prompting

In [6]:
if LLM_to_PDDL:

    pddl_system_prompt = "You are generating PDDL files for robot planning."\
        " When generating domain actions, do not include variables and keep the parameters empty."\
        " Write predicates using the original name of the object as in the dictionary."\
        " If an object name contains a whitespace, replace it with an underscore."\
        " You should also use negations where applicable. If something is \"empty\", say that it contains \"air\"."\
        " Use the following predicates:\n"\
        "(:predicates\n"\
        "   (in ?obj_1 - object ?obj_2 - object) -- this means ?obj_2 is in ?obj_1\n"\
        "   (on ?obj_1 - object ?obj_2 - object) -- this means ?obj_2 is on ?obj_1\n"\
        "   (under ?obj_1 - object ?obj_2 - object) -- this means ?obj_2 is under ?obj_1\n\n" \
        "   (is-whole ?obj_1 - object) (is-diced ?obj_1 - object) (is-chopped ?obj_1 - object) (is-sliced ?obj_1 - object) (is-mixed ?obj_1 - object)"\
        " (is-ground ?obj_1 - object) (is-juiced ?obj_1 - object) (is-spread ?obj_1 - object) )"
        # "\n\nUse the following example:\n"\
        # "(:action pour_water\n"\
        # " :parameters ()\n"\
        # " :precondition (and\n"\
        # "   (in cup water)\n"\
        # "   (in bowl air)\n"\
        # " )\n"\
        # " :effect (and "\
        # "   (in bowl water)\n"\
        # "   (in cup air)\n"\
        # "   (not (in cup water))\n"\
        # "   (not (in bowl air))\n"\
        # " )"\
        # ")"

    message = [{"role": "system", "content": pddl_system_prompt}]

    # for x in range(len(OLP_results['OLP']['Instructions'])):
    pddl_user_prompt = f"Generate PDDL for :\n{llm_output['OLP']['Instructions']}"

    message.extend([{"role": "user", "content": pddl_user_prompt}])

    _, response = prompt_LLM(given_prompt=message, model_name=openai_models[-1], verbose=False)
    print(response)


(:action pick_and_place_first_white_toy
 :parameters ()
 :precondition (and (on first_white_toy table) (in white_box air))
 :effect (and (in first_white_toy white_box) (not (on first_white_toy table)) (not (in white_box air))))

(:action pick_and_place_second_white_toy
 :parameters ()
 :precondition (and (on second_white_toy table) (in white_box first_white_toy))
 :effect (and (in second_white_toy white_box) (not (on second_white_toy table))))

(:action pick_and_place_first_green_toy
 :parameters ()
 :precondition (and (on first_green_toy table) (in green_box air))
 :effect (and (in first_green_toy green_box) (not (on first_green_toy table)) (not (in green_box air))))

(:action pick_and_place_second_green_toy
 :parameters ()
 :precondition (and (on second_green_toy table) (in green_box first_green_toy))
 :effect (and (in second_green_toy green_box) (not (on second_green_toy table))))

(:action pick_and_place_third_green_toy
 :parameters ()
 :precondition (and (on third_green_toy table)

### Writing FOON to a Text File

In [7]:
# -- save the prototype FOON graph as a text file, which we will then run with a parser to correct numbering:
if not os.path.exists('preprocess/'):
	os.makedirs('preprocess/')

if not os.path.exists('postprocess/'):
	os.makedirs('postprocess/')

file_ = open('preprocess/prototype.txt', 'w')
file_.write('#\tFOON Prototype\n#\t-- Task Prompt: {0}\n//\n'.format(user_task))
for unit in FOON_prototype:
	unit.print_functions[2]()
	file_.write(unit.getFunctionalUnitText())
	print('//')

file_.close()

O	first white toy
S	on	 [table]
O	white box
S	empty
M	Pick and Place	<Assumed>
O	first white toy
S	in	 [white box]
O	white box
S	contains	{first white toy}
//
O	second white toy
S	on	 [table]
O	white box
S	contains	{first white toy}
M	Pick and Place	<Assumed>
O	second white toy
S	in	 [white box]
O	white box
S	contains	{first white toy,second white toy}
//
O	first green toy
S	on	 [table]
O	green box
S	empty
M	Pick and Place	<Assumed>
O	first green toy
S	in	 [green box]
O	green box
S	contains	{first green toy}
//
O	second green toy
S	on	 [table]
O	green box
S	contains	{first green toy}
M	Pick and Place	<Assumed>
O	second green toy
S	in	 [green box]
O	green box
S	contains	{first green toy,second green toy}
//
O	third green toy
S	on	 [table]
O	green box
S	contains	{first green toy,second green toy}
M	Pick and Place	<Assumed>
O	third green toy
S	in	 [green box]
O	green box
S	contains	{first green toy,second green toy,third green toy}
//


## Step 2 :- FOON to PDDL
1. Parse FOON file -- this step is important to ensure that all labels are unique and that the generated file follows the FOON syntax.

2. (Optional) Visualize FOON graph

3. Run ``FOON_to_PDDL.py`` script to generate FOON macro-operators

### Parse and clean generated FOON

In [8]:
# -- running parsing module to ensure that FOON labels and IDs are made consistent for further use:
#		(it is important that each object and state type have a *UNIQUE* identifier)
fpa.skip_JSON_conversion = True		# -- we don't need JSON versions of a FOON
fpa.skip_index_check = True			# -- always create a new set of index files

fpa.source_dir = './preprocess/'
fpa.target_dir = './postprocess/'
fpa._run_parser()

-- [FOON_parser] : Initiating parsing procedure!


 -- [FOON_parser] : Commencing parsing...
  -- parsing 'prototype.txt'...

 -- [FOON_parser] : Saving corrected files to './postprocess/'...
  -- Saving 'prototype.txt'...

-- Revising object label senses using WordNet and Concept-Net...


SUMMARY OF CHANGES:
  -- new total of OBJECTS : 8
  -- new total of STATES : 4
  -- new total of MOTIONS : 1

 -- [FOON_parser] : Parsing complete!


### (Optional) Run FOON visualization tool

In [9]:
# -- after running the parser, take the 'prototype.txt' file and plot it using the following tool: https://davidpaulius.github.io/foon-view/
# fga._startFOONview()

### Generate PDDL files using ```FOON_to_PDDL``` module

In [10]:
import FOON_TAMP as fta
from pathlib import Path

FOON_subgraph_file = './postprocess/prototype.txt'

# -- definition of macro and micro plan file names:
macro_plan_file = os.path.splitext(FOON_subgraph_file)[0] + '_macro.plan'
micro_plan_file = os.path.splitext(FOON_subgraph_file)[0] + '_micro.plan'

fta.micro_domain_file = './FOON_skills.pddl'

# -- create a new folder for the generated problem files and their corresponding plans:
fta.micro_problems_dir = './micro_problems-' + Path(FOON_subgraph_file).stem
if not os.path.exists(fta.micro_problems_dir):
    os.makedirs(fta.micro_problems_dir)

ftp = fta.ftp

# -- perform conversion of the FOON subgraph file to PDDL:
ftp.FOON_subgraph_file = FOON_subgraph_file
ftp._convert_to_PDDL('OCP')

## -- parse through the newly created domain file and find all (macro) planning operators:
all_macro_POs = {PO.name : PO for PO in fta._parseDomainPDDL(ftp.FOON_domain_file)}

# -- checking to see if a macro level plan has been found (FD allows exporting to a file):
outcome = fta._findPlan(
    domain_file=ftp.FOON_domain_file,   # NOTE: FOON_to_PDDL object will already contain the names of the
    problem_file=ftp.FOON_problem_file,  #   corresponding macro-level domain and problem files
    output_plan_file=macro_plan_file
)

# TODO: write planning pipeline for other planners?

complete_micro_plan = []

if fta._checkPlannerOutput(output=outcome):
    print("  -- [FOON-TAMP] : A macro-level plan for '" + str(FOON_subgraph_file) + "' was found!")

    # -- now that we have a plan, we are going to parse it for the steps:
    macro_file = open(macro_plan_file, 'r')

    # NOTE: counters for macro- and micro-level steps:
    macro_count, micro_count = 0, 0

    macro_plan_lines = list(macro_file)
    for L in macro_plan_lines:
        if L.startswith('('):
            print('\t\t\t' + str(macro_count) + ' : ' + L.strip())
    print()

    for L in range(len(macro_plan_lines)):

        if macro_plan_lines[L].startswith('('):
            # NOTE: this is where we have identified a macro plan's step; here, we check the contents of its PO definition for:
            #	1. preconditions - this will become a sub-problem file's initial states (as predicates)
            #	2. effects - this will become a sub-problem file's goal states (as predicates)

            macro_count += 1

            macro_PO_name = macro_plan_lines[L][1:-2].strip()

            print(" -- [FOON-TAMP] : Searching for micro-level plan for '" + macro_PO_name + "' macro-PO...")

            # -- try to find this step's matching planning operator definition:
            matching_PO_obj = all_macro_POs[macro_PO_name] if macro_PO_name in all_macro_POs else None

            # -- when we find the equivalent planning operator, then we proceed to treat it as its own problem:
            if matching_PO_obj:
                # -- create sub-problem file (i.e., at the micro-level):
                micro_problem_file = fta._defineMicroProblem(
                    macro_PO_name,
                    preconditions=matching_PO_obj.getPreconditions(),
                    effects=matching_PO_obj.getEffects(),
                )

                micro_domain_file = fta._defineMicroDomain(
                    preconditions=matching_PO_obj.getPreconditions(),
                )

                complete_micro_plan.append('; step ' + str(macro_count) + ' -- (' + macro_PO_name + '):')

                need_to_replan = False

                # -- try to find a sub-problem plan / solution:
                outcome = fta._findPlan(
                    domain_file=micro_domain_file,
                    problem_file=micro_problem_file,
                    output_plan_file=str(fta.micro_problems_dir + '/' + macro_PO_name + '_micro.plan')
                )

                print('\n\t' + 'step ' + str(macro_count) +' -- (' + macro_PO_name + ')')

                if fta._checkPlannerOutput(outcome):

                    print('\t\t -- micro-level plan found as follows:')

                    # -- open the micro problem file, read each line referring to a micro PO, and save to list:
                    micro_file = open(
                        str(fta.micro_problems_dir + '/' + macro_PO_name + '_micro.plan'), 'r')

                    # -- all except for the last line should be valid steps:
                    micro_file_lines = []
                    count = 0

                    for micro_line in micro_file:
                        if micro_line.startswith('('):
                            # -- parse the line and remove trailing newline character:
                            micro_step = micro_line.strip()
                            micro_file_lines.append(micro_step)

                            # -- print entire plan to the command line in format of X.Y,
                            #       where X is the macro-step count and Y is the micro-step count:
                            count += 1
                            print('\t\t\t' + str(macro_count) + '.' + str(count) + ' : ' + micro_step)
                        #endif
                    #endfor
        print()
else:
    print("  -- [FOON-TAMP] : A macro-level plan for '" + str(FOON_subgraph_file) + "' was NOT found!")


  -- Reading file line : 100%|██████████| 51/51 [00:00<00:00, 8341.18it/s]


< FOON_TAMP: converting FOON graph to PDDL planning problem (last updated: 31st March, 2023)>


 -- [FOON-fga] : Opening FOON file named './postprocess/prototype.txt'...
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- 

  -- [FOON-TAMP] : A macro-level plan for './postprocess/prototype.txt' was found!
			0 : (pick_and_place_2 )
			0 : (pick_and_place_3 )
			0 : (pick_and_place_4 )

 -- [FOON-TAMP] : Searching for micro-level plan for 'pick_and_place_2' macro-PO...

	step 1 -- (pick_and_place_2)
		 -- micro-level plan found as follows:
			1.1 : (move robot robot table)
			1.2 : (pick first_green_toy table)
			1.3 : (move robot table green_box)
			1.4 : (insert first_green_toy green_box)

 -- [FOON-TAMP] : Searching for micro-level plan for 'pick_and_place_3' macro-PO...

	step 2 -- (pick_and_place_3)
		 -- micro-level plan found as follows:
			2.1 : (move robot robot table)
			2.2 : (pick second_green_toy table)
			2.3 : (move robot table green_box)
			2.4 : (insert second_green_toy green_box)

 -- [FOON-TAMP] : Searching for micro-level plan for 'pick_and_place_4' macro-PO...

	step 3 -- (pick_and_place_4)
		 -- micro-level plan found as follows:
			3.1 : (move robot robot table)
			3.2 : (pick third_

In [11]:
complete_micro_plan = []

# -- we can also perform a search where we assume we already have a task tree
#		and we execute each functional unit one by one as we see

macro_plan_lines = []
for N in range(len(ftp.fga.FOON_lvl3)):
	PO_name = f'{ftp._reviseObjectLabels(ftp.fga.FOON_lvl3[N].getMotion().getMotionLabel())}_{N}'
	macro_plan_lines.append(PO_name)

macro_count = 0

for L in range(len(macro_plan_lines)):

	macro_count += 1

	macro_PO_name = macro_plan_lines[L]

	print(" -- [FOON-TAMP] : Searching for micro-level plan for '" + macro_PO_name + "' macro-PO...")

	# -- try to find this step's matching planning operator definition:
	matching_PO_obj = all_macro_POs[macro_PO_name] if macro_PO_name in all_macro_POs else None

	# -- when we find the equivalent planning operator, then we proceed to treat it as its own problem:
	if matching_PO_obj:
		# -- create sub-problem file (i.e., at the micro-level):
		micro_problem_file = fta._defineMicroProblem(
			macro_PO_name,
			preconditions=matching_PO_obj.getPreconditions(),
			effects=matching_PO_obj.getEffects(),
		)

		micro_domain_file = fta._defineMicroDomain(
			preconditions=matching_PO_obj.getPreconditions(),
		)

		complete_micro_plan.append('; step ' + str(macro_count) + ' -- (' + macro_PO_name + '):')

		need_to_replan = False

		# -- try to find a sub-problem plan / solution:
		outcome = fta._findPlan(
			domain_file=micro_domain_file,
			problem_file=micro_problem_file,
			output_plan_file=str(fta.micro_problems_dir + '/' + macro_PO_name + '_micro.plan')
		)

		print('\n\t' + 'step ' + str(macro_count) +' -- (' + macro_PO_name + ')')

		if fta._checkPlannerOutput(outcome):

			print('\t\t -- micro-level plan found as follows:')

			# -- open the micro problem file, read each line referring to a micro PO, and save to list:
			micro_file = open(
				str(fta.micro_problems_dir + '/' + macro_PO_name + '_micro.plan'), 'r')

			# -- all except for the last line should be valid steps:
			micro_file_lines = []
			count = 0

			for micro_line in micro_file:
				if micro_line.startswith('('):
					# -- parse the line and remove trailing newline character:
					micro_step = micro_line.strip()
					micro_file_lines.append(micro_step)

					# -- print entire plan to the command line in format of X.Y,
					#       where X is the macro-step count and Y is the micro-step count:
					count += 1
					print('\t\t\t' + str(macro_count) + '.' + str(count) + ' : ' + micro_step)
				#endif
			#endfor
	print()


 -- [FOON-TAMP] : Searching for micro-level plan for 'pick_and_place_0' macro-PO...

	step 1 -- (pick_and_place_0)
		 -- micro-level plan found as follows:
			1.1 : (move robot robot table)
			1.2 : (pick first_white_toy table)
			1.3 : (move robot table white_box)
			1.4 : (insert first_white_toy white_box)

 -- [FOON-TAMP] : Searching for micro-level plan for 'pick_and_place_1' macro-PO...

	step 2 -- (pick_and_place_1)
		 -- micro-level plan found as follows:
			2.1 : (move robot robot table)
			2.2 : (pick second_white_toy table)
			2.3 : (move robot table white_box)
			2.4 : (insert second_white_toy white_box)

 -- [FOON-TAMP] : Searching for micro-level plan for 'pick_and_place_2' macro-PO...

	step 3 -- (pick_and_place_2)
		 -- micro-level plan found as follows:
			3.1 : (move robot robot table)
			3.2 : (pick first_green_toy table)
			3.3 : (move robot table green_box)
			3.4 : (insert first_green_toy green_box)

 -- [FOON-TAMP] : Searching for micro-level plan for 'pick_and_pl